In [2]:
using Rx
using BenchmarkTools

In [3]:
struct KeepActor{D} <: NextActor{D} 
    values::Vector{D}
    
    KeepActor{D}() where D = new(Vector{D}())
end

Rx.on_next!(actor::KeepActor{D}, data::D) where D = push!(actor.values, data)

# Map operator performance comparison

In [4]:
function regularArrayMap()
    source = [ i for i in 1:1000 ]
    keep   = Vector{Int}()
    for v in source
        push!(keep, v ^ 2)
    end
    return keep
end

@CreateMapOperator(SquaredInt, Int, Int, (d) -> d ^ 2)

function rxArrayGeneratedMap()
    values = Rx.from([ i for i in 1:1000 ])
    keep = KeepActor{Int}()
    subscribe!(values |> SquaredIntMapOperator(), keep)
    return keep.values
end

function rxArrayLambdaMap()
    values = Rx.from([ i for i in 1:1000 ])
    keep = KeepActor{Int}()
    subscribe!(values |> map(Int, (d) -> d ^ 2), keep)
    return keep.values
end

println(regularArrayMap() == rxArrayGeneratedMap())
println(regularArrayMap() == rxArrayLambdaMap())

@btime regularArrayMap();
@btime rxArrayGeneratedMap();
@btime rxArrayLambdaMap();

true
true
  7.066 μs (11 allocations: 24.33 KiB)
  7.074 μs (15 allocations: 24.41 KiB)
  76.873 μs (2483 allocations: 62.98 KiB)


# Filter operator performance comparison

In [5]:
function regularArrayFilter()
    source = [ i for i in 1:1000 ]
    keep   = Vector{Int}()
    for v in source
        if v % 2 == 0
            push!(keep, v)
        end
    end
    return keep
end

@CreateFilterOperator(EvenInt, Int, (d) -> d % 2 == 0)

function rxArrayGeneratedFilter()
    values = Rx.from([ i for i in 1:1000 ])
    keep = KeepActor{Int}()
    subscribe!(values |> EvenIntFilterOperator(), keep)
    return keep.values
end

function rxArrayLambdaFilter()
    values = Rx.from([ i for i in 1:1000 ])
    keep = KeepActor{Int}()
    subscribe!(values |> filter((d) -> d % 2 == 0), keep)
    return keep.values
end

println(regularArrayFilter() == rxArrayGeneratedFilter())
println(regularArrayFilter() == rxArrayLambdaFilter())

@btime regularArrayFilter();
@btime rxArrayGeneratedFilter();
@btime rxArrayLambdaFilter();

true
true
  4.627 μs (10 allocations: 16.27 KiB)
  4.588 μs (14 allocations: 16.34 KiB)
  48.611 μs (1504 allocations: 39.64 KiB)


# Count operator performance comparison

In [6]:
function regularArrayCount()
    source = [ i for i in 1:1000 ]
    keep   = Vector{Int}()
    current = 0
    for v in source
        current += 1
    end
    push!(keep, current)
    return keep
end

function rxArrayCount()
    values = Rx.from([ i for i in 1:1000 ])
    keep = KeepActor{Int}()
    subscribe!(values |> count(), keep)
    return keep.values
end

println(regularArrayCount() == rxArrayCount())

@btime regularArrayCount();
@btime rxArrayCount();

true
  656.950 ns (3 allocations: 8.06 KiB)
  919.400 ns (7 allocations: 8.16 KiB)


# Enumerate operator performance comparison

In [7]:
function regularArrayEnumerate()
    source = [ i for i in 1:1000 ]
    keep   = Vector{Tuple{Int, Int}}()
    current = 1
    for v in source
        push!(keep, (v, current))
        current += 1
    end
    return keep
end

function rxArrayEnumerate()
    values = Rx.from([ i for i in 1:1000 ])
    keep = KeepActor{Tuple{Int, Int}}()
    subscribe!(values |> enumerate(), keep)
    return keep.values
end

println(regularArrayEnumerate() == rxArrayEnumerate())

@btime regularArrayEnumerate();
@btime rxArrayEnumerate();

true
  7.735 μs (11 allocations: 40.34 KiB)
  8.606 μs (15 allocations: 40.44 KiB)


# Scan operator performance comparison

In [8]:
function regularArrayScan()
    source  = [ i for i in 1:1000 ]
    keep    = Vector{Int}()
    current = 0
    for v in source
        current = current + v
        push!(keep, current)
    end
    return keep
end

@CreateScanOperator(Sum, Int, Int, +)

function rxArrayGeneratedScan()
    values = Rx.from([ i for i in 1:1000 ])
    keep = KeepActor{Int}()
    subscribe!(values |> SumScanOperator(0), keep)
    return keep.values
end

function rxArrayLambdaScan()
    values = Rx.from([ i for i in 1:1000 ])
    keep = KeepActor{Int}()
    subscribe!(values |> scan(Int, (d, c) -> d + c, 0), keep)
    return keep.values
end

println(regularArrayScan() == rxArrayGeneratedScan())
println(regularArrayScan() == rxArrayLambdaScan())

@btime regularArrayScan();
@btime rxArrayGeneratedScan();
@btime rxArrayLambdaScan();

true
true
  6.703 μs (11 allocations: 24.33 KiB)
  7.925 μs (15 allocations: 24.42 KiB)
  994.808 μs (10273 allocations: 216.00 KiB)


# Reduce operator performance comparison

In [9]:
function regularArrayReduce()
    source  = [ i for i in 1:1000 ]
    keep    = Vector{Int}()
    current = 0
    for v in source
        current = current + v
    end
    push!(keep, current)
    return keep
end

@CreateReduceOperator(Sum, Int, Int, +)

function rxArrayGeneratedReduce()
    values = Rx.from([ i for i in 1:1000 ])
    keep = KeepActor{Int}()
    subscribe!(values |> SumReduceOperator(0), keep)
    return keep.values
end

function rxArrayLambdaReduce()
    values = Rx.from([ i for i in 1:1000 ])
    keep = KeepActor{Int}()
    subscribe!(values |> reduce(Int, (d, c) -> d + c, 0), keep)
    return keep.values
end

println(regularArrayReduce() == rxArrayGeneratedReduce())
println(regularArrayReduce() == rxArrayLambdaReduce())

@btime regularArrayReduce();
@btime rxArrayGeneratedReduce();
@btime rxArrayLambdaReduce();

true
true
  707.336 ns (3 allocations: 8.06 KiB)
  1.075 μs (7 allocations: 8.16 KiB)
  967.256 μs (9287 allocations: 184.45 KiB)


# Max operator performance comparison

In [10]:
function regularArrayMax()
    source  = [ i for i in 1:1000 ]
    keep    = Vector{Int}()
    current = nothing
    for v in source
        if current == nothing
            current = v
        else
            if v > current
                current = v
            end
        end
    end
    push!(keep, current)
    return keep
end

function rxArrayMax()
    values = Rx.from([ i for i in 1:1000 ])
    keep = KeepActor{Union{Int, Nothing}}()
    subscribe!(values |> max(), keep)
    return keep.values
end

println(regularArrayMax() == rxArrayMax())

@btime regularArrayMax();
@btime rxArrayMax();

true
  2.052 μs (3 allocations: 8.06 KiB)
  2.868 μs (9 allocations: 8.20 KiB)


# Min operator performance comparison

In [11]:
function regularArrayMin()
    source  = [ i for i in 1:1000 ]
    keep    = Vector{Int}()
    current = nothing
    for v in source
        if current == nothing
            current = v
        else
            if v < current
                current = v
            end
        end
    end
    push!(keep, current)
    return keep
end

function rxArrayMin()
    values = Rx.from([ i for i in 1:1000 ])
    keep = KeepActor{Union{Int, Nothing}}()
    subscribe!(values |> min(), keep)
    return keep.values
end

println(regularArrayMin() == rxArrayMin())

@btime regularArrayMin();
@btime rxArrayMin();

true
  2.990 μs (3 allocations: 8.06 KiB)
  2.722 μs (9 allocations: 8.20 KiB)


# Sum operator performance comparison

In [12]:
function regularArraySum()
    source  = [ i for i in 1:1000 ]
    keep    = Vector{Int}()
    current = nothing
    for v in source
        if current == nothing
            current = v
        else
            current = current + v
        end
    end
    push!(keep, current)
    return keep
end

function rxArraySum()
    values = Rx.from([ i for i in 1:1000 ])
    keep = KeepActor{Int}()
    subscribe!(values |> sum(), keep)
    return keep.values
end

println(regularArraySum() == rxArraySum())

@btime regularArraySum();
@btime rxArraySum();

true
  1.429 μs (3 allocations: 8.06 KiB)
  2.417 μs (9 allocations: 8.20 KiB)
